# Assignment 1

1.- Use the gpio.ipynb from lab as a starting point. Add a C++ function to reset all the GPIO pins on the chosen PMOD.

In [ ]:
from pynq.overlays.base import BaseOverlay
#import time
from time import sleep
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //instance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}
//Function to reset all GPIOs of PMODB
void reset_gpio_PMODB_all(){
    pyprintf("Reset all GPIOs of PMODB");
    for (int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}

In [ ]:
reset_gpio_PMODB_all()

2.- Write a Python3 cell that emulates a PWM (for a chosen frequency and duty cycle) on one of the PMOD GPIO pins. For example, PMODB PIN3 needs to turn on for duty_cycle percent of the square wave frequency and off for the rest of the square wave. It may not be possible to achieve exactly 0% or 100% so be sure to add necessary code for those corner cases.

In [ ]:
# Single PWM for any period and duty cycle 1 to 99%
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
from pynq.lib import Pmod_PWM
import time
# globals
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0

def pmw_gen(period,duty):
    period = min(max(period,1),65535)
    duty = min(max(duty,1),99)
    pwm.generate(period,duty)
    
pwm = Pmod_PWM(base.PMODB,PIN_0)    
pmw_gen(10,50)   
time.sleep(20)
pwm.stop()

In [ ]:
# Reset RED & BLUE leds, so we can vary the BLUE one
reset_gpio_PMODB_all()

3. Find the optimal PWM frequency, such that the physical flashing phenonmenon will not be perceived visually;

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //instance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}
//Function to reset all GPIOs of PMODB
void reset_gpio_PMODB_all(){
    pyprintf("Reset all GPIOs of PMODB");
    for (int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}

In [ ]:
import asyncio
threads_flag = True    # "cond" will keep threads live or kill them all
# Initial states
pmwR_flag = False
pmwG_flag = False
pmwB_flag = False
delay = 1;period = 1;duty = 25
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0
period=10;duty=90;on=0.5;off=0.5;blinks=0;blink_stop=100000
def user_input():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    ret = read_gpio(PIN_R);
    ret = read_gpio(PIN_G)
    ret = read_gpio(PIN_B)
    write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()
    default = 100;val1 = input("  Enter Frequency(Hz): {default}") or default
    freq = min(max(int(val1),1),65535)
    period = 1/freq# T=1/f
    print("period = "+ str(period))
    default = 25;val2 =  input("Enter Duty percentage: {default}") or default
    duty = min(max(int(val2),1),99)
    print("duty = "+ str(duty))
    on = (duty/100)*period
    off = period-on
    print("on="+str(on)+", off="+str(off))       
async def RGB_leds():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    while threads_flag:        
        if blinks >= blink_stop: 
            pmwR_flag = pmwG_flag = pmwB_flag = False
            blinks = 0
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()         
        await asyncio.sleep(off)
        if (pmwR_flag == True):
            write_gpio(PIN_R, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0)            
            blinks += 1
        elif (pmwG_flag == True):
            write_gpio(PIN_G, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_G, 0)
            blinks += 1
        elif (pmwB_flag == True):
            write_gpio(PIN_B, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_B, 0)           
            blinks += 1
        else:
            await asyncio.sleep(1)
            print('.', end='')
async def get_btns(_loop):
    btns = base.btns_gpio
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, delay, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0 # We specify global variables (not locals)
    while threads_flag:
        await asyncio.sleep(0.1)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("button 0 pressed, Enter params for RED")
                pmwR_flag = True;pmwG_flag = False;pmwB_flag = False
                user_input()
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("button 1 pressed, Enter params for GREEN")
                pmwG_flag = True;pmwR_flag = False;pmwB_flag = False 
                user_input()
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("button 2 pressed, Enter params for BLUE")
                pmwB_flag = True;pmwR_flag = False;pmwG_flag = False
                user_input()
            else:
                await asyncio.sleep(0.3)
                print("button 3 pressed, Exit program")
                threads_flag = False
                _loop.stop()        
### main sequence ##
write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); 
time.sleep(0.2)
loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(RGB_leds());   # 
loop.create_task(get_btns(loop)); # take user input buttons
print("Press buttons 0-3\n   0 - RED,\n   1 - GREEN\n   2 - BLUE\n   3 - Exit program")
loop.run_forever()
loop.close()        
print("Done.")

4. Achieve the visually perceived 100%, 75%, 50% and 25% of full LED brightness by adjusting the duty cycle

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //instance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}
//Function to reset all GPIOs of PMODB
void reset_gpio_PMODB_all(){
    pyprintf("Reset all GPIOs of PMODB");
    for (int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}

In [ ]:
import asyncio
threads_flag = True    # "cond" will keep threads live or kill them all
# Initial states
pmwR_flag = False
pmwG_flag = False
pmwB_flag = False
delay = 1;period = 1;duty = 25
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0
period=110;duty=90;on=0.5;off=0.5;blinks=0;blink_stop=100000
def user_input():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    ret = read_gpio(PIN_R);
    ret = read_gpio(PIN_G)
    ret = read_gpio(PIN_B)
    write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()
    default = 110;val1 = input("  Enter Frequency(Hz): {default}") or default
    freq = min(max(int(val1),1),65535)
    period = 1/freq# T=1/f
    print("period = "+ str(period))
    default = 25;val2 =  input("Enter Duty percentage: {default}") or default
    duty = min(max(int(val2),1),99)
    print("duty = "+ str(duty))
    on = (duty/100)*period
    off = period-on
    print("on="+str(on)+", off="+str(off))       
async def RGB_leds():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    while threads_flag:        
        if blinks >= blink_stop: 
            pmwR_flag = pmwG_flag = pmwB_flag = False
            blinks = 0
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()         
        await asyncio.sleep(off)
        if (pmwR_flag == True):
            write_gpio(PIN_R, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0)            
            blinks += 1
        elif (pmwG_flag == True):
            write_gpio(PIN_G, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_G, 0)
            blinks += 1
        elif (pmwB_flag == True):
            write_gpio(PIN_B, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_B, 0)           
            blinks += 1
        else:
            await asyncio.sleep(1)
            print('.', end='')
async def get_btns(_loop):
    btns = base.btns_gpio
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, delay, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0 # We specify global variables (not locals)
    while threads_flag:
        await asyncio.sleep(0.1)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("button 0 pressed, Enter params for RED")
                pmwR_flag = True;pmwG_flag = False;pmwB_flag = False
                user_input()
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("button 1 pressed, Enter params for GREEN")
                pmwG_flag = True;pmwR_flag = False;pmwB_flag = False 
                user_input()
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("button 2 pressed, Enter params for BLUE")
                pmwB_flag = True;pmwR_flag = False;pmwG_flag = False
                user_input()
            else:
                await asyncio.sleep(0.3)
                print("button 3 pressed, Exit program")
                threads_flag = False
                _loop.stop()        
### main sequence ##
write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); 
time.sleep(0.2)
loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(RGB_leds());   # 
loop.create_task(get_btns(loop)); # take user input buttons
print("Press buttons 0-3\n   0 - RED,\n   1 - GREEN\n   2 - BLUE\n   3 - Exit program")
loop.run_forever()
loop.close()        
print("Done.")

5. 5. Varying the duty cycles and approximate the corresponding LED brightness (in the unit of %). To the end, plot and explain the approximate relationship of % brightness versus duty cycle.

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //instance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}
//Function to reset all GPIOs of PMODB
void reset_gpio_PMODB_all(){
    pyprintf("Reset all GPIOs of PMODB");
    for (int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}

In [ ]:
import asyncio
threads_flag = True    # "cond" will keep threads live or kill them all
# Initial states
pmwR_flag = False
pmwG_flag = False
pmwB_flag = False
delay = 1;period = 1;duty = 0
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0
period=110;duty=90;on=0.5;off=0.5;blinks=0;blink_stop=100000
def user_input():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
    write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()
    default = 110;val1 = input("  Enter Frequency(Hz): {default}") or default
    freq = min(max(int(val1),1),65535)
    period = 1/freq# T=1/f
    print("period = "+ str(period))
    duty = 0
    print("Start Duty Cycle = "+ str(duty))
    
async def RGB_leds():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    while threads_flag:        
        if blinks >= blink_stop: 
            pmwR_flag = pmwG_flag = pmwB_flag = False
            blinks = 0
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()         
        await asyncio.sleep(off)
        if (pmwR_flag == True):
            write_gpio(PIN_R, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0)            
            blinks += 1
        elif (pmwG_flag == True):
            write_gpio(PIN_G, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_G, 0)
            blinks += 1
        elif (pmwB_flag == True):
            write_gpio(PIN_B, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_B, 0)           
            blinks += 1
        else:
            await asyncio.sleep(1)
            print('.', end='')
async def DutyCycleIncreasing():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    while threads_flag:
        if pmwR_flag == True or pmwG_flag == True or pmwB_flag == True:
            if duty > 100:
                duty = 0
                pmwR_flag = False;pmwG_flag = False;pmwB_flag = False
            else:
                duty += 5
                await asyncio.sleep(0.5)
                duty = min(max(int(duty),0),100)
                on = (duty/100)*period
                off = period-on
                print(str(duty)+" ", end='')
        else:
            await asyncio.sleep(0.5)
            
async def get_btns(_loop):
    btns = base.btns_gpio
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, delay, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0 # We specify global variables (not locals)
    while threads_flag:
        await asyncio.sleep(0.1)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("button 0 pressed, Enter params for RED")
                pmwR_flag = True;pmwG_flag = False;pmwB_flag = False
                user_input()
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("button 1 pressed, Enter params for GREEN")
                pmwG_flag = True;pmwR_flag = False;pmwB_flag = False 
                user_input()
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("button 2 pressed, Enter params for BLUE")
                pmwB_flag = True;pmwR_flag = False;pmwG_flag = False
                user_input()
            else:
                await asyncio.sleep(0.3)
                print("button 3 pressed, Exit program")
                threads_flag = False
                _loop.stop()        
### main sequence ##
write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); 
time.sleep(0.2)
loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(RGB_leds());   #
loop.create_task(DutyCycleIncreasing());   #
loop.create_task(get_btns(loop)); # take user input buttons

print("Press buttons 0-3\n   0 - RED,\n   1 - GREEN\n   2 - BLUE\n   3 - Exit program")
loop.run_forever()
loop.close()        
print("Done.")

In [ ]:
# Start here
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //intance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}

In [ ]:
from pynq.overlays.base import BaseOverlay
import pynq.lib.rgbled as rgbled 
import time
# globals
base = BaseOverlay("base.bit")
import asyncio
led4 = rgbled.RGBLED(4)
led5 = rgbled.RGBLED(5)
threads_flag = True    # "cond" will keep threads live or kill them all
# Initial states
pmwR_flag = False
pmwG_flag = False
pmwB_flag = False
duty = 25
# PMW
from pynq.lib import Pmod_PWM
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0
period=10;duty=90
# Coroutines:
async def RGB_leds():
    global threads_flag, pwmR, pwmG, pwmB, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)

    while threads_flag:
        if (pmwR_flag == True):
            pwmR = Pmod_PWM(base.PMODB,PIN_R)  
            pwmR.generate(period,duty)
            write_gpio(PIN_G, 0);write_gpio(PIN_B, 0)
            pmwR_flag = False
            await asyncio.sleep(0.1)
        elif (pmwG_flag == True):
            pwmG = Pmod_PWM(base.PMODB,PIN_G)
            pwmG.generate(period,duty)
            write_gpio(PIN_R, 0);write_gpio(PIN_B, 0)
            pmwG_flag = False
            await asyncio.sleep(0.1)
        elif (pmwB_flag == True):
            pwmB = Pmod_PWM(base.PMODB,PIN_B)
            pwmB.generate(period,duty)
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0)
            pmwB_flag = False
            await asyncio.sleep(0.1)
        else:
            await asyncio.sleep(0.1)

async def get_btns(_loop):
    btns = base.btns_gpio
    global threads_flag, pwmR, pwmG, pwmB, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0 # We specify global variables (not locals)
    while threads_flag:
        await asyncio.sleep(0.01)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("buton 0 pressed, RED")
                pmwR_flag = True
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("buton 1 pressed, GREEN")
                pmwG_flag = True
                
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("buton 2 pressed, BLUE")
                pmwB_flag = True

            else:
                await asyncio.sleep(0.3)
                print("buton 3 pressed, Exit program") 
                threads_flag = False
                pwmR.stop()
                pwmG.stop()
                pwmB.stop()             
                _loop.stop()        
### main sequence ##
pwmR = Pmod_PWM(base.PMODB,PIN_R)
pwmG = Pmod_PWM(base.PMODB,PIN_G)
pwmB = Pmod_PWM(base.PMODB,PIN_B)
time.sleep(0.5)
pwmR.stop()
pwmG.stop()
pwmB.stop()
write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);
loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(RGB_leds());   # 
loop.create_task(get_btns(loop)); # take user input
print("Press buttons 0-3\n   0 - Start binking,\n   1 - Stop\n   2 - No action\n   3 - Exit program")
loop.run_forever()
loop.close()        
led4.write(0x0)
led5.write(0x0)
print("Done.")

6. After you have experimented with various PWM frequencies and duty cycles, add the following:

<X> Functionalities for a fixed duty cycle (i.e. 25%). You’ll want to use asyncio for this part.
    
<X> Start the code and blink the LED’s red channel in intervals of 1 second (i.e. 1 second on, 1
second off)
    
<X> When buttons 0, 1, or 2 are pushed, the LED will change color from Red, to Green, to Blue.
    
<X> When button 3 is pushed, the LED will stop blinking.
    
<X> Your video should demonstrate how each button change to each of the (3) colors.

In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);        //instance object
    gpio_set_direction(pin_in, GPIO_IN); //configure to be Input
    return gpio_read(pin_in);
}
//Function to reset all GPIOs of PMODB
void reset_gpio_PMODB_all(){
    pyprintf("Reset all GPIOs of PMODB");
    for (int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin);
        gpio_set_direction(pin_out, GPIO_OUT);
        gpio_write(pin_out, 0);
    }
}

In [3]:
import asyncio
threads_flag = True    # "cond" will keep threads live or kill them all
# Initial states
pmwR_flag = True
pmwG_flag = False
pmwB_flag = False
delay = 1;period = 0.5;duty = 50
PIN_R = 3;PIN_G = 2;PIN_B = 1;PIN_0 = 0
on=1;off=1;blinks=0;blink_stop=100000
async def RGB_leds():
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, on, off, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0  # We specify global variables (not locals)
    while threads_flag:   
        if blinks >= blink_stop: 
            pmwR_flag = pmwG_flag = pmwB_flag = False
            blinks = 0
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0); #reset_gpio_PMODB_all()         
        await asyncio.sleep(off)
        if (pmwR_flag == True):
            write_gpio(PIN_R, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);          
            blinks += 1
        elif (pmwG_flag == True):
            write_gpio(PIN_G, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);
            blinks += 1
        elif (pmwB_flag == True):
            write_gpio(PIN_B, 1)
            await asyncio.sleep(on)
            ret = read_gpio(PIN_R)
            write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);       
            blinks += 1
        else:
            await asyncio.sleep(1)
            print('.', end='')
async def get_btns(_loop):
    btns = base.btns_gpio
    globals()['write_gpio']=write_gpio
    global threads_flag, blinks, blink_stop, delay, pmwR_flag, pmwG_flag, pmwB_flag, pmw_flag, period, duty, PIN_R, PIN_G, PIN_B, PIN_0 # We specify global variables (not locals)
    while threads_flag:
        await asyncio.sleep(0.1)
        if btns.read() != 0:
            if (btns[0].read()==1):
                await asyncio.sleep(0.3)
                print("button 0 pressed, Enter params for RED")
                ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
                write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);
                pmwR_flag = True;pmwG_flag = False;pmwB_flag = False
            elif (btns[1].read()==1):                
                await asyncio.sleep(0.3)
                print("button 1 pressed, Enter params for GREEN")
                ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
                write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);                
                pmwG_flag = True;pmwR_flag = False;pmwB_flag = False 
            elif (btns[2].read()==1):
                await asyncio.sleep(0.3)
                print("button 2 pressed, Enter params for BLUE")
                ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
                write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);                
                pmwB_flag = True;pmwR_flag = False;pmwG_flag = False
            else:
                await asyncio.sleep(0.3)
                print("button 3 pressed, Exit program")
                threads_flag = False
                await asyncio.sleep(1)
                ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
                write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);
                _loop.stop()        
### main sequence ##
ret = read_gpio(PIN_R);ret = read_gpio(PIN_G);ret = read_gpio(PIN_B)
write_gpio(PIN_R, 0);write_gpio(PIN_G, 0);write_gpio(PIN_B, 0);
freq = 0.5
period = 1/freq# T=1/f
print("period = "+ str(period))
print("duty = "+ str(duty))
on = (duty/100)*period
off = period-on
print("on="+str(on)+", off="+str(off))  
loop = asyncio.new_event_loop();  # Instance event_loop object
loop.create_task(RGB_leds());   # 
loop.create_task(get_btns(loop)); # take user input buttons
print("Press buttons 0-3\n   0 - RED,\n   1 - GREEN\n   2 - BLUE\n   3 - Exit program")
loop.run_forever()
loop.close()        
print("Done.")

period = 2.0
duty = 50
on=1.0, off=1.0
Press buttons 0-3
   0 - RED,
   1 - GREEN
   2 - BLUE
   3 - Exit program
button 0 pressed, Enter params for RED
button 1 pressed, Enter params for GREEN
button 1 pressed, Enter params for GREEN
button 2 pressed, Enter params for BLUE
button 1 pressed, Enter params for GREEN
button 0 pressed, Enter params for RED
button 3 pressed, Exit program
Done.
